# 1/ Env Setup
Load necessary libraries to run this notebook. <br>
All libraries are cited in ```requirements.txt```. <br>
Documentation: https://docs.pytorch.org/vision/main/models/generated/torchvision.models.detection.retinanet_resnet50_fpn_v2.html

## 1.1/ Import dependencies
Load libraries:

In [1]:
import sys  
print(sys.executable)

/Users/litani/Documents/myCode/steel-defects/venv-CVsteel/bin/python


In [2]:
import os
from pathlib import Path
import torch 
import torchvision
from torchvision.models.detection import retinanet_resnet50_fpn_v2
from torchvision.models.detection.retinanet import RetinaNetClassificationHead
import albumentations as A # note that albumentationsx was installed and but you still albumentations
from albumentations.pytorch import ToTensorV2
import numpy as np

## 1.2/ Set reproducibility
Device and seed:

In [3]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
torch.manual_seed(42)
np.random.seed(42)

# 2/ Configuration Management
Define:
- image path
- model hyperparameters
- hardware


In [6]:
class Config:
    # Paths
    DATA_ROOT = Path("../data/raw")
    TRAIN_IMG = DATA_ROOT / "train_images"
    TRAIN_ANN = DATA_ROOT / "train_annotations"
    TEST_IMG = DATA_ROOT / "valid_images"
    TEST_ANN = DATA_ROOT / "valid_annotations"

    # Model parameters
    NUM_CLASSES = 7 # 6 defects + 1 background
    BACKBONE_PRETRAINED = True 
    
    # Training hyperparameters
    BATCH_SIZE = 16  # no mention of batch size in the paper
    NUM_EPOCHS = 5 # 24 epochs based on paper. Reduced for quicker testing
    LEARNING_RATE = 0.0025 # 0.0025 based on paper
    MOMENTUM = 0.9 # 0.9 based on paper

    # Hardware
    DEVICE = device
    NUM_WORKERS = 8

config = Config()

In [7]:
config

# 3/ Dataset Class
- Load images and annotations into PyTorch format. 
- This is necessary since RetineNet excepts a dictionary format. The latter requires XML parsing.

In [13]:
sys.path.append("src")

In [14]:
from utils.parse_xml import parse_xml

ModuleNotFoundError: No module named 'utils'